<a href="https://colab.research.google.com/github/victor-roris/ML-learning/blob/master/ComputerVision/DeepLearning_ComputerVision_ImageClassification_PyTorch_DataSpartan_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MultiLabel Image Classification 

In this notebook we generate a model in `pytorch` to try to classify images of figure (from a **non public dataset**).

Adapted from [link](https://medium.com/@thevatsalsaglani/multi-class-image-classification-using-cnn-over-pytorch-and-the-basics-of-cnn-fdf425a11dc0) and [link](https://github.com/vatsalsaglani/ApparelClassifier/blob/master/train.py)

In [22]:
# Importing modules 
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
import shutil
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import models
from torchvision import transforms
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

## Dataset

### Download data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataset_folderpath = "/content/input"

In [4]:
if os.path.isdir(dataset_folderpath):
  !rm -R {dataset_folderpath}

In [5]:
from pathlib import Path
def mkdir(sfolderpath):
  folderpath = Path(sfolderpath)
  folderpath.mkdir(parents=True, exist_ok=True)

mkdir(dataset_folderpath)

In [6]:
!tar -xvf /content/drive/My\ Drive/DATASPARTAN/PROYECTOS/AI-DOCUMENTS/SEGMENTATION/DSFigureDataset.tar.gz -C {dataset_folderpath}

clear_output()

!ls {dataset_folderpath}

raw  setup.py


### Split `train`, `val` and `test` dataset

The `zip` contains a script to generate the ml dataset

In [7]:
% cd {dataset_folderpath}
!python setup.py

/content/input
Number of classes : 20
['LineGraph', 'BubbleChart', 'BoxPlot', 'PersonPhoto', 'VennDiagram', 'NetworkDiagram', 'GeneralPhoto', 'ParetoChart', 'ScatterGraph', 'Table', 'GeneralFigure', 'TreeDiagram', 'Signatures', 'Map', 'RadarPlot', 'AreaGraph', 'PieChart', 'FlowChart', 'HorizontalBarGraph', 'VerticalBarGraph']
LineGraph: 
   > ntrain : 91
   > nval : 37
   > ntest : 2

BubbleChart: 
   > ntrain : 65
   > nval : 28
   > ntest : 1

BoxPlot: 
   > ntrain : 109
   > nval : 46
   > ntest : 2

PersonPhoto: 
   > ntrain : 76
   > nval : 31
   > ntest : 2

VennDiagram: 
   > ntrain : 100
   > nval : 42
   > ntest : 2

NetworkDiagram: 
   > ntrain : 67
   > nval : 28
   > ntest : 1

GeneralPhoto: 
   > ntrain : 42
   > nval : 18
   > ntest : 1

ParetoChart: 
   > ntrain : 97
   > nval : 40
   > ntest : 2

ScatterGraph: 
   > ntrain : 70
   > nval : 29
   > ntest : 1

Table: 
   > ntrain : 75
   > nval : 31
   > ntest : 2

GeneralFigure: 
   > ntrain : 111
   > nval : 46
   > nte

In [8]:
input_folder = "/content/input/dataset"

trn_folder = os.path.join(input_folder,"train")
val_folder = os.path.join(input_folder,"val")
test_folder = os.path.join(input_folder,"test")

In [9]:
classes = [name for name in os.listdir(trn_folder) 
      if os.path.isdir(os.path.join(trn_folder,name))]
print(f"Number of classes : {len(classes)}")
print(classes)

Number of classes : 20
['LineGraph', 'BubbleChart', 'BoxPlot', 'PersonPhoto', 'VennDiagram', 'NetworkDiagram', 'GeneralPhoto', 'ParetoChart', 'ScatterGraph', 'Table', 'GeneralFigure', 'TreeDiagram', 'Signatures', 'Map', 'RadarPlot', 'AreaGraph', 'PieChart', 'FlowChart', 'HorizontalBarGraph', 'VerticalBarGraph']


In [10]:
# Output of script
total_test=38

### Data preparation

Train and Val files are distributed in each folder with subfolders by class:

```
 folder\
    cls1\
        image.png
        image2.jpeg
        image3.png
        ...
    cls2\
        imageA.jpeg
        imageB.jpg
        imageC.png
        ...
    ...
```

In [11]:
IMG_HEIGHT = 512
IMG_WIDTH = 512
batch_size = 2

In [12]:
# create dataloaders
tfms = transforms.Compose([transforms.Resize((IMG_HEIGHT,IMG_WIDTH)), 
                           transforms.ToTensor(),
                           transforms.Normalize([0.458, 0.456, 0.406],
                                                [0.229, 0.224, 0.225])])

In [13]:
train = ImageFolder(trn_folder, tfms)
valid = ImageFolder(val_folder, tfms)

In [14]:
train_data_loader = DataLoader(train, batch_size=batch_size, num_workers=4)
valid_data_loader = DataLoader(valid, batch_size=batch_size, num_workers=4)

## Model

In [16]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.ConvLayer1 = nn.Sequential(
            nn.Conv2d(3, 8, 3), # inp (3, 512, 512)
            nn.Conv2d(8, 16, 3),
            nn.MaxPool2d(2),
            nn.ReLU() # op (16, 256, 256)
        )
        self.ConvLayer2 = nn.Sequential(
            nn.Conv2d(16, 32, 5), # inp (16, 256, 256)
            nn.Conv2d(32, 32, 3),
            nn.MaxPool2d(4),
            nn.ReLU() # op (32, 64, 64)
        )
        self.ConvLayer3 = nn.Sequential(
            nn.Conv2d(32, 64, 3), # inp (32, 64, 64)
            nn.Conv2d(64, 64, 5),
            nn.MaxPool2d(2),
            nn.ReLU() # op (64, 32, 32)
        )
        self.ConvLayer4 = nn.Sequential(
            nn.Conv2d(64, 128, 5), # inp (64, 32, 32)
            nn.Conv2d(128, 128, 3),
            nn.MaxPool2d(2),
            nn.ReLU() # op (128, 16, 16)
        )
        self.Lin1 = nn.Linear(15488, 20) # 15488 = 512x512 : Resize all images to 512x512  # Len(classes) = 20
        self.Lin2 = nn.Linear(1500, 150)
        self.Lin3 = nn.Linear(150, 20) # Len(classes) = 20
        
        
    def forward(self, x):
        x = self.ConvLayer1(x)
        x = self.ConvLayer2(x)
        x = self.ConvLayer3(x)
        x = self.ConvLayer4(x)
        x = x.view(x.size(0), -1)
        x = self.Lin1(x)
       
        
        return F.log_softmax(x, dim = 1)

In [17]:
model = Net()

In [19]:
if torch.cuda.is_available():
  model.cuda()
else:
  print("GPU is not available. The trainning process could be really slow.")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.5)
for epoch in tqdm(range(50)):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 10 == 0:    # print every 10 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0

print('Finished Training')

## Validation

In [23]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_data_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 12 %
